# I am  going to show you how to filter RDD and dataframe using its row numbers

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Download_Data___STOCK_US_XNYS_WMT.csv,Download_Data___STOCK_US_XNYS_WMT.csv,1390
dbfs:/FileStore/tables/VS14MORT_txt.gz,VS14MORT_txt.gz,93093001
dbfs:/FileStore/tables/employee-1.txt,employee-1.txt,1023
dbfs:/FileStore/tables/employee.csv,employee.csv,28031
dbfs:/FileStore/tables/employee.txt,employee.txt,1023
dbfs:/FileStore/tables/international_migration_March_2021_estimated_migration_by_age_sex.csv,international_migration_March_2021_estimated_migration_by_age_sex.csv,4464659
dbfs:/FileStore/tables/test-1.csv,test-1.csv,335
dbfs:/FileStore/tables/test-2.csv,test-2.csv,335
dbfs:/FileStore/tables/test.csv,test.csv,335
dbfs:/FileStore/tables/test.xml,test.xml,292


In [0]:
rdd = sc.textFile("dbfs:/FileStore/tables/employee.txt")

In [0]:
rdd.count()

Out[4]: 21

In [0]:
rdd.take(3)

Out[5]: ['Emp_id,Last_name,designation,job_id,hire_date,base_salary,commission,increment_pct',
 '7369,SMITH,CLERK,7902,17-Dec-80,800,\\000,20',
 '7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30']

In [0]:
rdd=rdd.zipWithIndex()

In [0]:
rdd.take(3)

Out[7]: [('Emp_id,Last_name,designation,job_id,hire_date,base_salary,commission,increment_pct',
  0),
 ('7369,SMITH,CLERK,7902,17-Dec-80,800,\\000,20', 1),
 ('7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30', 2)]

# you can see row number in each row from the above output.

In [0]:
rdd.filter(lambda x:x[1]==11).collect()

Out[8]: [('7876,ADAMS,CLERK,7788,12-Jan-83,1100,\\000,20', 11)]

# we have filter out 11th row from an RDD

# lets filter the data frame

In [0]:
df = spark.read.csv("dbfs:/FileStore/tables/employee.txt", header=True,inferSchema=True)

In [0]:
df.display()

Emp_id,Last_name,designation,job_id,hire_date,base_salary,commission,increment_pct
7369,SMITH,CLERK,7902,17-Dec-80,800,\000,20
7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30
7521,WARD,SALESMAN,7698,22-Feb-81,1250,500,30
7566,JONES,MANAGER,7839,02-Apr-81,2975,\000,20
7654,MARTIN,SALESMAN,7698,28-Sep-81,1250,1400,30
7698,BLAKE,MANAGER,7839,01-May-81,2850,\000,30
7782,CLARK,MANAGER,7839,09-Jun-81,2450,\000,10
7788,SCOTT,ANALYST,7566,09-Dec-82,3000,\000,20
7839,KING,PRESIDENT,\000,17-Nov-81,5000,\000,10
7844,TURNER,SALESMAN,7698,08-Sep-81,1500,0,30


lets filter out the rows using row_number function

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [0]:
spec=Window.orderBy("Emp_ID")

In [0]:
df=df.withColumn("row_num",row_number().over(spec))

In [0]:
df.display()

Emp_id,Last_name,designation,job_id,hire_date,base_salary,commission,increment_pct,row_num
7333,ALAN,CLERK,7783,30-Apr-80,1600,\000,10,1
7342,MAYIM,ANALYST,7566,30-Apr-80,3150,\000,10,2
7369,SMITH,CLERK,7902,17-Dec-80,800,\000,20,3
7432,FROST,MANAGER,7839,15-Sep,2600,\000,30,4
7499,ALLEN,SALESMAN,7698,20-Feb-81,1600,300,30,5
7521,WARD,SALESMAN,7698,22-Feb-81,1250,500,30,6
7566,JONES,MANAGER,7839,02-Apr-81,2975,\000,20,7
7654,MARTIN,SALESMAN,7698,28-Sep-81,1250,1400,30,8
7698,BLAKE,MANAGER,7839,01-May-81,2850,\000,30,9
7730,NAOMI,CLERK,7902,01-Jan-80,1000,\000,20,10


# you can see row number column from the above df

In [0]:
df.filter(df.row_num==11).show()

+------+---------+-----------+------+---------+-----------+----------+-------------+-------+
|Emp_id|Last_name|designation|job_id|hire_date|base_salary|commission|increment_pct|row_num|
+------+---------+-----------+------+---------+-----------+----------+-------------+-------+
|  7782|    CLARK|    MANAGER|  7839|09-Jun-81|       2450|      \000|           10|     11|
+------+---------+-----------+------+---------+-----------+----------+-------------+-------+



# thats all my friends. Thanks a lot for watching this video.